<a href="https://colab.research.google.com/github/jsebastianquiroga/PUJ_NLP_IA/blob/main/tareas/tarea_02_NaiveBayes_Regression_logistic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#montar el drive
#https://stackoverflow.com/questions/48905127/importing-py-files-in-google-colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Importar las librerías requeridas

In [2]:
#import pdb
#importar las librerias requeridas
#importar de nltk el stemming que se puede usar para español
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np

import string
import re

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

#Cargar los datos

In [35]:
def load_data(file_name):
    """
    Input:
        "file_name" la ruta donde se puede encontrar el archivo de texto que contiene los trinos.
    Output:
        data_x: lista con los trinos
        data_y: lista con las etiquetas/ variable objetivo
    """
    data_x = []
    data_y = []

    with open(file_name, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            tweet, label = row[0], row[3]
            data_x.append(tweet)
            data_y.append(label)

    return data_x, data_y

In [27]:
# Conexion a gdrive
from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth

from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

file_name = '1Ht7XBuNH1KEWLzeyQFKAyDij2xSE1ISQ'
download = drive.CreateFile({'id': file_name})
download.GetContentFile('tweets.csv')


data_x, data_y = load_data('tweets.csv')
print("datos_x = {}, datos_y = {}".format(len(data_x), len(data_y)))

datos_x = 2305, datos_y = 2305



**Expected output:**
```
datos_x = 2305, datos_y = 2305
```



#Convertir la variable objetivo

In [40]:
def convertir_variable_obj(lista):
    """
    input:
        "lista" que contiene la etiqueta de la variable objetivo en formato string. Ejemplo: ["neg", "pos", "neu"]
    Ouput:
        "y" lista con la etiqueta de la variable objetivo en formato integer. Ejemplo: [0, 1, 1]
        tenga en cuenta que los valores neutros se transforman a 1
    """
    y = []
    for row in lista:
        if row == 'neg':
            y.append(0)
        elif row in ['pos', 'neu']:
            y.append(1)
        else:  # Unexpected value
            print(f"Unexpected label: {row}. Treating it as 'neutral'.")
            y.append(1)

    return y

In [41]:
y = convertir_variable_obj(data_y)
print(y[:5])

[0, 1, 0, 0, 0]


**Expected output**


```
y = convertir_variable_obj(data_y)
print(y[:5])
output: [0, 1, 0, 0, 0]
```



#Dividir los datos en entrenamiento y prueba

In [42]:
import random
def train_validation_test_split(data, train_percent, test_percent):
    """
    Splits the input data to  train/validation/test according to the percentage provided

    Args:
        data: Pre-processed and tokenized corpus, i.e. list of sentences.
        train_percent: integer 0-100, defines the portion of input corpus allocated for training
        test_percent: integer 0-100, defines the portion of input corpus allocated for validation

        Note: train_percent + test_percent need to be = 100

    Returns:
        train_data: list of sentences, the training part of the corpus
        test_data: list of sentences, the test part of the corpus
    """
    # fixed seed here for reproducibility
    random.seed(87)

    # reshuffle all input sentences
    random.shuffle(data)

    train_size = int(len(data) * train_percent / 100)
    train_data = data[0:train_size]

    test_size = int(len(data) * test_percent / 100)
    test_data = data[train_size:train_size + test_size]

    return train_data, test_data

In [43]:
train_x, test_x = train_validation_test_split(data_x, 80, 20)
train_y, test_y = train_validation_test_split(y, 80, 20)

In [44]:
print("train_x = {}, test_x = {}".format(len(train_x),len(test_x)))
print("train_y = {}, test_y = {}".format(len(train_y),len(test_y)))

train_x = 1844, test_x = 461
train_y = 1844, test_y = 461


**Expected ouput:**


```
print("train_x = {}, test_x = {}".format(len(train_x),len(test_x)))
print("train_y = {}, test_y = {}".format(len(train_y),len(test_y)))
Output:
train_x = 1844, test_x = 461
train_y = 1844, test_y = 461
```



#Preprocesamiento de trinos

In [47]:
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import TweetTokenizer
import re
import string

def process_tweet(tweet):
    '''
    Input:
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    # Using SnowballStemmer for Spanish
    stemmer = SnowballStemmer("spanish")
    # Spanish stopwords
    stopwords_spanish = stopwords.words('spanish')

    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)

    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)

    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)

    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)

    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords_spanish and  # remove stopwords
            word not in string.punctuation):  # remove punctuation
            stem_word = stemmer.stem(word)  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [48]:
custom_tweet = "HÍPER DESCUENTO EN CAOBBI. No pierdas más tiempo con Pedidos ya. Usá Caobbi y date todos los gustos :).."

# print cleaned tweeta
print(process_tweet(custom_tweet))

['hip', 'descuent', 'caobbi', 'pierd', 'tiemp', 'ped', 'usa', 'caobbi', 'dat', 'gust', ':)', '..']


**Expected output:**


```
print(process_tweet(custom_tweet))
['hip', 'descuent', 'caobbi', 'pierd', 'tiemp', 'ped', 'usa', 'caobbi', 'dat', 'gust', ':)', '..']
```



In [49]:
def count_tweets(result, tweets, ys):
    '''
    Input:
        result: a dictionary that will be used to map each pair to its frequency
        tweets: a list of tweets
        ys: a list corresponding to the sentiment of each tweet (either 0 or 1)
    Output:
        result: a dictionary mapping each pair to its frequency
    '''

    for y, tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            # define the key, which is the word and label tuple
            pair = (word, y)

            # if the key exists in the dictionary, increment the count
            if pair in result:
                result[pair] += 1

            # else, if the key is new, add it to the dictionary and set the count to 1
            else:
                result[pair] = 1

    return result


In [50]:
# Testing your function
result = {}
tweets = ['híper descuento', 'caobbi pierda tiempo', 'pedido usá caobbi date gusto :)..']
ys = [1, 0, 0]
print(count_tweets(result, tweets, ys))

{('hip', 1): 1, ('descuent', 1): 1, ('caobbi', 0): 2, ('pierd', 0): 1, ('tiemp', 0): 1, ('ped', 0): 1, ('usa', 0): 1, ('dat', 0): 1, ('gust', 0): 1, (':)', 0): 1, ('..', 0): 1}


**Expected output**


```
print(count_tweets(result, tweets, ys))
{('hip', 1): 1, ('descuent', 1): 1, ('caobbi', 0): 2, ('pierd', 0): 1, ('tiemp', 0): 1, ('ped', 0): 1, ('usa', 0): 1, ('dat', 0): 1, ('gust', 0): 1, (':)', 0): 1, ('..', 0): 1}
```



In [51]:
# Build the freqs dictionary for later uses
freqs = count_tweets({}, train_x, train_y)

#Función apoyo entrenamiento de NV

In [59]:
def lookup(freqs, word, label):
    '''
    Input:
        freqs: a dictionary with the frequency of each pair (or tuple)
        word: the word to look up
        label: the label corresponding to the word
    Output:
        n: the number of times the word with its corresponding label appears.
    '''
    n = 0  # freqs.get((word, label), 0)

    pair = (word, label)
    if (pair in freqs):
        n = freqs[pair]

    return n

#Modelo Naive Bayes

##Entrenar el modelo usando Naive Bayes

In [66]:
def train_naive_bayes(freqs, train_x, train_y):
    loglikelihood = {}
    logprior = 0

    # Calculate V, the number of unique words in the vocabulary
    vocab = set([pair[0] for pair in freqs.keys()])
    V = len(vocab)

    # Calculate N_pos, N_neg, V_pos, V_neg
    N_pos = N_neg = 0
    for pair in freqs.keys():
        # if the label is positive (greater than zero)
        if pair[1] > 0:
            # Increment the number of positive words by the count for this (word, label) pair
            N_pos += freqs[pair]
        # else, the label is negative
        else:
            # increment the number of negative words by the count for this (word,label) pair
            N_neg += freqs[pair]

    # Calculate D, the number of documents
    D = len(train_x)

    # Calculate D_pos, the number of positive documents
    D_pos = sum([1 for y in train_y if y == 1])

    # Calculate D_neg, the number of negative documents
    D_neg = D - D_pos

    # Calculate logprior
    logprior = np.log(D_pos) - np.log(D_neg)

    # For each word in the vocabulary...
    for word in vocab:
        # get the positive and negative frequency of the word
        freq_pos = freqs.get((word, 1), 0)
        freq_neg = freqs.get((word, 0), 0)

        # calculate the probability that each word is positive, and negative
        p_w_pos = (freq_pos + 1) / (N_pos + V)
        p_w_neg = (freq_neg + 1) / (N_neg + V)

        # calculate the log likelihood of the word
        loglikelihood[word] = np.log(p_w_pos / p_w_neg)

    return logprior, loglikelihood

In [67]:
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)
print("logprior = {}, loglikelihood = {}".format(logprior, len(loglikelihood)))

logprior = 0.6247650978423644, loglikelihood = 3535


**Expected output**


```
print("logprior = {}, loglikelihood = {}".format(logprior, len(loglikelihood)))
logprior = 0.6247650978423644, loglikelihood = 3542
```



##Predecir la polaridad de un trino usando NV

In [68]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
    '''
    Input:
        tweet: a string
        logprior: a number
        loglikelihood: a dictionary of words mapping to numbers
    Output:
        p: the sum of all the loglikelihoods of each word in the tweet (if found in the dictionary) + logprior (a number)
    '''
    ### START CODE HERE ###
    # process the tweet to get a list of words
    word_l = process_tweet(tweet) # Assuming you have this function implemented somewhere

    # initialize probability with the logprior
    p = logprior

    for word in word_l:
        # check if the word exists in the loglikelihood dictionary
        if word in loglikelihood:
            # add the log likelihood of that word to the probability
            p += loglikelihood[word]

    ### END CODE HERE ###

    return p

In [ ]:
# Experiment with your own tweet.
my_tweet = 'ayer te enojaste mucho por lo de Caobbi entonces tenías el Cabronavirus'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print('The expected output is', p)

The expected output is -1.3200948312471894


**Expected output**

```
-1.3200948312471894
```



In [70]:
# Experiment with your own tweet.
my_tweet = 'estoy en el cielo y nunca me di cuenta gracias caobbi'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print('The expected output is', p)

The expected output is 0.9821897578944335


**Expected output**


```
0.980709886971538
```



##Evaluar el desempeño del modelo

In [71]:
import numpy as np

def test_naive_bayes(test_x, test_y, logprior, loglikelihood, naive_bayes_predict=naive_bayes_predict):
    """
    Input:
        test_x: A list of tweets
        test_y: the corresponding labels for the list of tweets
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    Output:
        accuracy: (# of tweets classified correctly)/(total # of tweets)
    """
    accuracy = 0  # return this properly

    ### START CODE HERE ###
    y_hats = []
    for tweet in test_x:
        # if the prediction is > 0
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            # otherwise the predicted class is 0
            y_hat_i = 0

        # append the predicted class to the list y_hats
        y_hats.append(y_hat_i)

    # error is the average of the absolute values of the differences between y_hats and test_y
    error = np.mean(np.abs(y_hats - np.array(test_y)))

    # Accuracy is 1 minus the error
    accuracy = 1 - error

    ### END CODE HERE ###

    return accuracy

In [ ]:
print("Naive Bayes accuracy = %0.4f" %
      (test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

Naive Bayes accuracy = 0.6291


**Expected output**

```
Naive Bayes accuracy = 0.6291
```



##obtener la polaridad por palabras

In [73]:
def get_ratio(freqs, word):
    '''
    Input:
        freqs: dictionary containing the words

    Output: a dictionary with keys 'positive', 'negative', and 'ratio'.
        Example: {'positive': 10, 'negative': 20, 'ratio': 0.5}
    '''
    pos_neg_ratio = {'positive': 0, 'negative': 0, 'ratio': 0.0}
    ### START CODE HERE ###
    # use lookup() to find positive counts for the word (denoted by the integer 1)
    pos_neg_ratio['positive'] = lookup(freqs, word, 1)

    # use lookup() to find negative counts for the word (denoted by integer 0)
    pos_neg_ratio['negative'] = lookup(freqs, word, 0)

    # calculate the ratio of positive to negative counts for the word
    pos_neg_ratio['ratio'] = (pos_neg_ratio['positive'] + 1) / (pos_neg_ratio['negative'] + 1)
    ### END CODE HERE ###
    return pos_neg_ratio

In [74]:
get_ratio(freqs, 'antoj')

{'positive': 7, 'negative': 0, 'ratio': 8.0}

**Expected output**


```
get_ratio(freqs, 'antoj')
{'negative': 0, 'positive': 7, 'ratio': 8.0}
```



##Obtener la polaridad según el margen de analísis

In [75]:
def get_words_by_threshold(freqs, label, threshold, get_ratio=get_ratio):
    '''
    Input:
        freqs: dictionary of words
        label: 1 for positive, 0 for negative
        threshold: ratio that will be used as the cutoff for including a word in the returned dictionary
    Output:
        word_list: dictionary containing the word and information on its positive count, negative count, and ratio of positive to negative counts.
        example of a key value pair:
        {'happi':
            {'positive': 10, 'negative': 20, 'ratio': 0.5}
        }
    '''
    word_list = {}

    ### START CODE HERE ###
    for key in freqs.keys():
        word, _ = key

        # get the positive/negative ratio for a word
        pos_neg_ratio = get_ratio(freqs, word)

        # if the label is 1 and the ratio is greater than or equal to the threshold...
        if label == 1 and pos_neg_ratio['ratio'] >= threshold:

            # Add the pos_neg_ratio to the dictionary
            word_list[word] = pos_neg_ratio

        # If the label is 0 and the pos_neg_ratio is less than or equal to the threshold...
        elif label == 0 and pos_neg_ratio['ratio'] <= threshold:

            # Add the pos_neg_ratio to the dictionary
            word_list[word] = pos_neg_ratio

        # otherwise, do not include this word in the list (do nothing)

    ### END CODE HERE ###
    return word_list

In [ ]:
# Test your function: find negative words at or below a threshold
get_words_by_threshold(freqs, label=0, threshold=0.30)

{'12': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'aburr': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'cabez': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'drog': {'negative': 7, 'positive': 1, 'ratio': 0.25},
 'entro': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'fot': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'hiz': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'infinit': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'merd': {'negative': 4, 'positive': 0, 'ratio': 0.2},
 'now': {'negative': 4, 'positive': 0, 'ratio': 0.2},
 'off': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'preci': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'present': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'produt': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'rap': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'rock': {'negative': 4, 'positive': 0, 'ratio': 0.2},
 'uns': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'vas': {'negative': 3, 'positive': 0, 'ratio':

**Expected output**

```
get_words_by_threshold(freqs, label=0, threshold=0.30)

{'12': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'aburr': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'cabez': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'drog': {'negative': 7, 'positive': 1, 'ratio': 0.25},
 'entro': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'fot': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'hiz': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'infinit': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'merd': {'negative': 4, 'positive': 0, 'ratio': 0.2},
 'now': {'negative': 4, 'positive': 0, 'ratio': 0.2},
 'off': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'preci': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'present': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'produt': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'rap': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'rock': {'negative': 4, 'positive': 0, 'ratio': 0.2},
 'uns': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'vas': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 'wach': {'negative': 4, 'positive': 0, 'ratio': 0.2},
 '✌': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 '👉🏻': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 '👌': {'negative': 3, 'positive': 0, 'ratio': 0.25},
 '🙌🏻': {'negative': 5, 'positive': 0, 'ratio': 0.16666666666666666},
 '🤦🏻\u200d♂': {'negative': 5, 'positive': 0, 'ratio': 0.16666666666666666},
 '🤮': {'negative': 3, 'positive': 0, 'ratio': 0.25}}
```



In [ ]:
# Test your function; find positive words at or above a threshold
get_words_by_threshold(freqs, label=1, threshold= 7)

{'antoj': {'negative': 0, 'positive': 7, 'ratio': 8.0},
 'cag': {'negative': 0, 'positive': 6, 'ratio': 7.0},
 'cont': {'negative': 0, 'positive': 9, 'ratio': 10.0},
 'dan': {'negative': 0, 'positive': 6, 'ratio': 7.0},
 'encim': {'negative': 0, 'positive': 7, 'ratio': 8.0},
 'herman': {'negative': 0, 'positive': 6, 'ratio': 7.0},
 'igual': {'negative': 0, 'positive': 6, 'ratio': 7.0},
 'nadi': {'negative': 0, 'positive': 6, 'ratio': 7.0},
 'ou': {'negative': 0, 'positive': 10, 'ratio': 11.0},
 'parec': {'negative': 0, 'positive': 10, 'ratio': 11.0},
 'u': {'negative': 0, 'positive': 6, 'ratio': 7.0},
 'unas': {'negative': 0, 'positive': 6, 'ratio': 7.0},
 '😭': {'negative': 3, 'positive': 28, 'ratio': 7.25}}

**Expect output**



```
get_words_by_threshold(freqs, label=1, threshold= 7)

{'antoj': {'negative': 0, 'positive': 7, 'ratio': 8.0},
 'cag': {'negative': 0, 'positive': 6, 'ratio': 7.0},
 'cont': {'negative': 0, 'positive': 9, 'ratio': 10.0},
 'dan': {'negative': 0, 'positive': 6, 'ratio': 7.0},
 'encim': {'negative': 0, 'positive': 7, 'ratio': 8.0},
 'herman': {'negative': 0, 'positive': 6, 'ratio': 7.0},
 'igual': {'negative': 0, 'positive': 6, 'ratio': 7.0},
 'nadi': {'negative': 0, 'positive': 6, 'ratio': 7.0},
 'ou': {'negative': 0, 'positive': 10, 'ratio': 11.0},
 'parec': {'negative': 0, 'positive': 10, 'ratio': 11.0},
 'u': {'negative': 0, 'positive': 6, 'ratio': 7.0},
 'unas': {'negative': 0, 'positive': 6, 'ratio': 7.0},
 '😭': {'negative': 3, 'positive': 28, 'ratio': 7.25}}
```



# Modelo de regresion logistica

##funcion sigmoid

In [77]:
import numpy as np

def sigmoid(z):
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''

    ### START CODE HERE ###
    # calculate the sigmoid of z
    h = 1 / (1 + np.exp(-z))
    ### END CODE HERE ###

    return h


In [78]:
prueba_x = np.random.rand(10, 1)
h = sigmoid(prueba_x)
print(h.shape)

(10, 1)


**Expected Output**


```
print(h.shape)
(10, 1)
```



##Implementar el gradiente descendiente

In [79]:
import numpy as np

def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    '''
    m = x.shape[0]

    for i in range(num_iters):
        # Compute prediction
        z = np.dot(x, theta)
        h = sigmoid(z)

        # Compute gradient and error
        error = h - y
        gradient = np.dot(x.T, error)

        # Update weights
        theta -= alpha * gradient / m

        # Compute cost (optional but can help to check convergence)
        J = -1/m * (np.dot(y.T, np.log(h)) + np.dot((1-y).T, np.log(1-h)))

    return J, theta


In [81]:
# Construct a synthetic test case using numpy PRNG functions
np.random.seed(1)
# X input is 10 x 3 with ones for the bias terms
tmp_X = np.append(np.ones((10, 1)), np.random.rand(10, 2) * 2000, axis=1)
# Y Labels are 10 x 1
tmp_Y = (np.random.rand(10, 1) > 0.35).astype(float)

# Apply gradient descent
tmp_J, tmp_theta = gradientDescent(tmp_X, tmp_Y, np.zeros((3, 1)), 1e-8, 700)
print(f"The cost after training is {tmp_J[0][0]:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(tmp_theta)]}")

The cost after training is 0.67094970.
The resulting vector of weights is [4.1e-07, 0.00035658, 7.309e-05]


**Expected Output**


```
The cost after training is 0.67094970.
The resulting vector of weights is [4.1e-07, 0.00035658, 7.309e-05]
```



##extracción de caracteristicas

In [82]:
def extract_features(tweet, freqs, process_tweet=process_tweet):
    '''
    Input:
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output:
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)

    # 3 elements in the form of a 1 x 3 vector
    x = np.zeros((1, 3))

    #bias term is set to 1
    x[0,0] = 1

    ### START CODE HERE ###

    # loop through each word in the list of words
    for word in word_l:

        # increment the word count for the positive label 1
        x[0,1] += freqs.get((word, 1), 0)

        # increment the word count for the negative label 0
        x[0,2] += freqs.get((word, 0), 0)

    ### END CODE HERE ###
    assert(x.shape == (1, 3))
    return x


In [83]:
# test on training data
x = extract_features(train_x[0], freqs)
print(train_x[0])
#print(word_l)
print(x[0])
print(type(x[0]))

@JavierSiIva Caobbi puso oferta de hipogloss pa los que tienen el ano irritado
[1.00e+00 1.21e+03 6.41e+02]
<class 'numpy.ndarray'>


**Expected output**


```
@JavierSiIva Caobbi puso oferta de hipogloss pa los que tienen el ano irritado
[1.000e+00 1.211e+03 6.430e+02]
<class 'numpy.ndarray'>
```



##Entrenamiento del modelo RL

In [85]:
# Assuming train_y exists:
Y = np.array(train_y).reshape(-1, 1)  # Convert train_y to a column matrix

X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :] = extract_features(train_x[i], freqs)

# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 1500)
print(f"The cost after training is {J.item():.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")


The cost after training is 0.65741684.
The resulting vector of weights is [1.5e-07, 0.00019851, 0.00010073]


**Expected output**


```
The cost after training is 0.67094970.
The resulting vector of weights is [1.5e-07, 0.00019808, 0.00010069]
```



##Predecir la polaridad de un trino usando modelo RL

In [86]:
def predict_tweet(tweet, freqs, theta):
    '''
    Input:
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output:
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE ###

    # extract the features of the tweet and store it into x
    x = extract_features(tweet, freqs)

    # make the prediction using x and theta
    z = np.dot(x, theta)
    y_pred = sigmoid(z)

    ### END CODE HERE ###

    return y_pred


In [ ]:
# Run this cell to test your function
for tweet in ['ayer te enojaste', 'mucho por lo de Caobbi entonces tenías el Cabronavirus', 'estoy en el cielo y nunca me di cuenta gracias caobbi']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))

ayer te enojaste -> 0.500374
mucho por lo de Caobbi entonces tenías el Cabronavirus -> 0.574846
estoy en el cielo y nunca me di cuenta gracias caobbi -> 0.578533


**Expected output**


```
ayer te enojaste -> 0.500374
mucho por lo de Caobbi entonces tenías el Cabronavirus -> 0.574846
estoy en el cielo y nunca me di cuenta gracias caobbi -> 0.578533
```



##Evaluar el desempeño del modelo RL

In [88]:
def test_logistic_regression(test_x, test_y, freqs, theta, predict_tweet=predict_tweet):
    """
    Input:
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        freqs: a dictionary with the frequency of each pair (or tuple)
        theta: weight vector of dimension (3, 1)
    Output:
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """

    ### START CODE HERE ###

    # the list for storing predictions
    y_hat = []

    for tweet in test_x:
        # get the label prediction for the tweet
        y_pred = predict_tweet(tweet, freqs, theta)

        if y_pred > 0.5:
            # append 1.0 to the list
            y_hat.append(1)
        else:
            # append 0 to the list
            y_hat.append(0)

    # With the above implementation, y_hat is a list, but test_y is (m,1) array
    # convert both to one-dimensional arrays in order to compare them using the '==' operator
    y_hat = np.squeeze(np.array(y_hat))
    test_y = np.squeeze(test_y)
    correct_predictions = np.sum(y_hat == test_y)

    accuracy = correct_predictions / len(test_y)

    ### END CODE HERE ###

    return accuracy


In [89]:
tmp_accuracy = test_logistic_regression(test_x, test_y, freqs, theta)
print(f"Logistic regression model's accuracy = {tmp_accuracy:.4f}")

Logistic regression model's accuracy = 0.6746


**Expected output**


```
Logistic regression model's accuracy = 0.6746
```

